### Set Up
 - https://parameter-sherpa.readthedocs.io/en/latest/gettingstarted/installation.html
 - pip install pymongo

これでOK

### jupyter notebookで
以下のパスが必要

- `cd /opt/conda/lib/python3.6/site-packages/`
- `ln -nfs /your/path/sherpa/sherpa sherpa`

In [1]:
import sys
import sherpa
print(sherpa.__file__)

/opt/conda/lib/python3.6/site-packages/sherpa/__init__.py


### まずは普通に

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

Using TensorFlow backend.


In [3]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784) # 2次元配列を1次元に変換
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')   # int型をfloat32型に変換
x_test = x_test.astype('float32')
x_train /= 255                        # [0-255]の値を[0.0-1.0]に変換
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 42s 4us/step
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s 158us/step - loss: 0.2452 - acc: 0.9248 - val_loss: 0.1047 - val_acc: 0.9667
Epoch 2/20
60000/60000 [==============================] - 9s 156us/step - loss: 0.1034 - acc: 0.9686 - val_loss: 0.0936 - val_acc: 0.9732
Epoch 3/20
60000/60000 [==============================] - 9s 156us/step - loss: 0.0745 - acc: 0.9771 - val_loss: 0.0780 - val_acc: 0.9784
Epoch 4/20
60000/60000 [==============================] - 9s 156us/step - loss: 0.0607 - acc: 0.9813 - val_loss: 0.0746 - val_acc: 0.9791
Epoch 5/20
60000/60000 [==============================] - 10s 165us/step - loss: 0.0510 - acc: 0.9849 - val_loss: 0.0668 - val_acc: 0.9820
Epoch 6/20
60000/60000 [==============================] - 10s 168us/step - loss: 0.0428 - acc: 0.9877 - val_loss: 0.0682 - val_acc: 0.9833
Epoch 7/20
60000/60000 [==============================] - 9s 157us/step - loss: 0.0386 - acc: 0.9887 - val_loss: 0.0856 

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.114885806221
Test accuracy: 0.982


### ここまでは普通のMNIST
以下でsherpaを利用してハイパーパラメータのチューニングを行う

In [8]:
import sherpa

parameters = [sherpa.Discrete('num_units', [25, 200]),
              sherpa.Continuous('dropout', [0., 0.5])]

alg = sherpa.algorithms.BayesianOptimization(num_grid_points=3, max_num_trials=4)

In [9]:
study = sherpa.Study(parameters=parameters,
                     algorithm=alg,
                     lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running on http://172.17.0.4:8880
-------------------------------------------------------
Process Process-2:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 938, in run
    cli.show_server_banner(self.env, self.debug, self.name, False)
  File "/opt/conda/lib/python3.6/site-packages/flask/cli.py", line 629, in show_server_banner
    click.echo(message)
  File "/opt/conda/lib/python3.6/site-packages/click/utils.py", line 259, in echo
    file.write(message)
io.UnsupportedOperation: not writable


In [10]:
for trial in study:
    print(trial.parameters)
    model = Sequential()
    model.add(Dense(units=int(trial.parameters['num_units']), activation='relu', input_shape=(784,)))
    model.add(Dropout(trial.parameters['dropout']))
    model.add(Dense(units=int(trial.parameters['num_units']), activation='relu'))
    model.add(Dropout(trial.parameters['dropout']))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    callbacks=[study.keras_callback(trial, objective_name='val_loss')],
                    validation_data=(x_test, y_test))
    study.finalize(trial)

{'dropout': 0.125, 'num_units': 68}
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 43us/step - loss: 0.4376 - acc: 0.8700 - val_loss: 0.1922 - val_acc: 0.9438
Epoch 2/5
60000/60000 [==============================] - 2s 30us/step - loss: 0.2084 - acc: 0.9375 - val_loss: 0.1361 - val_acc: 0.9599
Epoch 3/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.1582 - acc: 0.9523 - val_loss: 0.1235 - val_acc: 0.9625
Epoch 4/5
60000/60000 [==============================] - 2s 40us/step - loss: 0.1389 - acc: 0.9583 - val_loss: 0.1039 - val_acc: 0.9691
Epoch 5/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.1209 - acc: 0.9629 - val_loss: 0.1012 - val_acc: 0.9695
{'dropout': 0.125, 'num_units': 112}
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.3577 - acc: 0.8951 - val_loss: 0.1605 - val_acc: 0.9512
Epoch 2/5
6

In [11]:
print(study.get_best_result())

{'Trial-ID': 2, 'Iteration': 4, 'dropout': 0.125, 'num_units': 112, 'Objective': 0.082121041838638484}


### この値を利用すればOK